In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup as bsp
import pandas as pd
import time, random

In [8]:
URL = "https://contrataciondelestado.es"

In [9]:
def waitFinishLoad(driver, byLoader=By.ID, valueLoader=None, dissapear=False, extra_wait=False):
    if valueLoader is None: return
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((byLoader, valueLoader)))
    # if dissapear: WebDriverWait(driver, 10).until(EC.invisibility_of_element((byLoader, valueLoader)))
    if extra_wait: time.sleep(random.choice([0.6, 0.5, 0.8, 0.7, 0.9, 0.4]))

def get_elem(driver, by, value):
    return driver.find_element(by, value)

def get_elements(driver, by, value):
    return driver.find_elements(by, value)

def click(driver, by, value, wait=True, byLoader=By.ID, valueLoader=None, extra_wait=False):
    if wait: waitFinishLoad(driver, byLoader, valueLoader, extra_wait)
    get_elem(driver, by, value).click()

def click_script(driver, value, by=By.ID, wait=True, byLoader=By.ID, valueLoader=None, extra_wait=False):
    if wait: waitFinishLoad(driver, byLoader, valueLoader, extra_wait)
    driver.execute_script(get_elem(driver, by, value).get_dom_attribute("onClick"))
    
def menu_busqueda(driver, menu, wait=True, byLoader=By.ID, valueLoader=None, extra_wait=True):
    for m in menu:
        if wait: waitFinishLoad(driver, byLoader, valueLoader=f'//*[contains(., "{m}")]', extra_wait=extra_wait)
        get_elem(driver, By.XPATH, f'//*[text() = "{m}"]/../../td/img[contains(@src, "plus")]').click()

def screenshot_expedientes(driver:webdriver.Firefox):
    expedientes = get_elements(driver, By.XPATH, value="//img[@title='Abre en pestaña nueva']")
    for exp in expedientes:
        exp.click()
        driver.switch_to.window(driver.window_handles[1])
        waitFinishLoad(driver, valueLoader="footer-newShow")
        cabecera = driver.find_elements(By.XPATH, value="//span[@class='outputText cabecera']")
        title = (cabecera[1].text).replace("/","_")
        driver.save_screenshot(f"images/Expediente_{title}.png")
        driver.close()
        driver.switch_to.window(driver.window_handles[0])

def get_table(driver:webdriver.Firefox):
    page = driver.find_elements(by=By.TAG_NAME, value="table")[-1]
    soup = bsp(page.get_attribute('innerHTML'), "html.parser")
    cols, rows = soup.find_all("thead"), soup.find_all("tbody")
    retdf = pd.read_html("<table>"+str(cols)+str(rows)+"</table>")[0]
    return retdf

In [10]:
GeckoDriverManager().install()
driver = webdriver.Firefox()

In [11]:
try:
    # # Ultimas licitaciones publicadas
    # driver.get(url=URL) 
    # waitFinishLoad(driver, valueLoader="footer-newShow", extra_wait=True)
    # # click(driver=driver, by=By.XPATH, value=f'//a[contains(@href, "licitaciones")]', valueLoader="footer-newShow")
    # click(driver, By.LINK_TEXT, value="Licitaciones", valueLoader="footer-newShow")

    # Pagina principal
    driver.get(url=URL) 
    waitFinishLoad(driver, valueLoader="footer-newShow", extra_wait=True) # Si la conexión es muy lenta puede dar fallo si no esperamos a que cargue la primera página
    
    # Pagina licitaciones
    click(driver=driver, by=By.XPATH, value=f'//a[contains(@href, "licitaciones")]', valueLoader="footer-newShow")
    
    # Formulario de Búsqueda
    click_script(driver=driver,by=By.PARTIAL_LINK_TEXT, value="licitaciones", byLoader=By.PARTIAL_LINK_TEXT, valueLoader="licitaciones")
    
    # Formulario de Búsqueda Avanzada
    click_script(driver=driver,by=By.PARTIAL_LINK_TEXT, value="avanzada", byLoader=By.PARTIAL_LINK_TEXT, valueLoader="avanzada")

    # Formulario de Búsqueda Avanzada Sellecionar
    click_script(driver=driver,by=By.PARTIAL_LINK_TEXT, value="Seleccionar", byLoader=By.PARTIAL_LINK_TEXT, valueLoader="Seleccionar")
    
    # Seleccionar
    menu = ["ENTIDADES LOCALES", "Galicia", "A Coruña", "Ayuntamientos"]
    menu_busqueda(driver, menu, byLoader=By.XPATH, extra_wait=True)

    # clickar coruña 2
    get_elements(driver, By.XPATH, f'//*[text() = "A Coruña"]')[-1].click()
    
    # clickar juntas
    click(driver=driver, by=By.XPATH, value=f'//option[contains(text(), "Junta de Gobierno del Ayuntamiento")]')
    
    # clickar añadir
    click(driver=driver, by=By.XPATH, value=f'//input[contains(@value, "Añadir")]')
    
    # click buscar
    click(driver=driver, by=By.XPATH, value=f'//input[contains(@value, "Buscar")]')
except NoSuchElementException as e:
    print("Error\n", e)

In [12]:
# blucle para extraer los datos de la tabla 
try:
    df = get_table(driver)
    next = True
    contador = 0
    while next:
        driver.save_screenshot(f"images/tabla_{contador}.png") 
        contador +=1
        screenshot_expedientes(driver)
        if get_elem(driver, by=By.XPATH, value=f'//input[contains(@value, "Siguiente")]'):
            click(driver,by=By.XPATH, value=f'//*[contains(@value, "Siguiente")]', valueLoader="footer-newShow")
            new_df = get_table(driver)
            df = pd.concat([df, new_df], ignore_index=True)
        else: next = False
except NoSuchElementException as e:
    print("No se ha encontrado el elemento Siguiente")

No se ha encontrado el elemento Siguiente


In [13]:
df.isna().sum()

Expediente                 0
Tipo de Contrato           0
Estado                     0
Importe                    0
Fechas                    11
Órgano de Contratación     0
dtype: int64

In [14]:
df

,Expediente,Tipo de Contrato,Estado,Importe,Fechas,Órgano de Contratación
0,521/2022/1025Obras de mejoras de accesibilidad...,ObrasConstrucción,Adjudicada,"573.986,91",Publicación PLACSP:Adjudicación:07/02/2024,Junta de Gobierno del Ayuntamiento de A Coruña
1,521/2021/449Procedimiento abierto simplificado...,ObrasConstrucción general de inmuebles y obras...,Adjudicada,"322.790,03",Publicación PLACSP:Adjudicación:07/02/2024,Junta de Gobierno del Ayuntamiento de A Coruña
2,521/2022/535Obras de mejora de accesibilidad m...,ObrasConstrucción,Adjudicada,"357.954,28",Publicación PLACSP:Adjudicación:07/02/2024,Junta de Gobierno del Ayuntamiento de A Coruña
3,"851/2023/101 L3 y L4Suministro, Servicios y Ob...",SuministrosAdquisición,Adjudicada,"2.122.314,05",Publicación PLACSP:Adjudicación:07/02/2024,Junta de Gobierno del Ayuntamiento de A Coruña
4,"851/2023/101Suministro, Servicios y Obras de E...",SuministrosAdquisición,Parcialmente Adjudicada,"2.122.314,05",Publicación PLACSP:Adjudicación:07/02/2024,Junta de Gobierno del Ayuntamiento de A Coruña
...,...,...,...,...,...,...
883,"370/2018/60Contrato de préstamo, con previa ap...",Privado,Adjudicada,"7.975.450,00",Publicación PLACSP:Adjudicación:15/06/2018,Junta de Gobierno del Ayuntamiento de A Coruña
884,106/2018/117Suministro de Gas Natural para cal...,SuministrosAdquisición,Evaluación,"28.099,17",Present. Oferta:28/06/2018,Junta de Gobierno del Ayuntamiento de A Coruña
885,236/2018/41Obra de sustitución de la cubierta ...,ObrasConstrucción,Resuelta,"58.162,82",Publicación PLACSP:Adjudicación:31/05/2018Form...,Junta de Gobierno del Ayuntamiento de A Coruña
886,541/2018/916Refuerzo de contenedores en la zon...,ServiciosServicios de alcantarillado y elimina...,Evaluación,"16.043,52",Present. Oferta:24/05/2018,Junta de Gobierno del Ayuntamiento de A Coruña


In [15]:
from sqlalchemy import URL, create_engine, text

connection_url  =  URL.create(
    drivername = "mssql+pyodbc",
    username = "sa",
    password = "Abcd1234.",
    host = "10.133.28.194",
    port = 41433,
    database = "master",
    query = {
        "driver" : "ODBC Driver 18 for SQL Server",
        "Encrypt" : "yes",
        "TrustServerCertificate" : "yes",
    }
)

engine = create_engine(connection_url)
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS Licitaciones"))
    conn.commit()

InterfaceError: (pyodbc.InterfaceError) ('IM002', '[IM002] [Microsoft][Administrador de controladores ODBC] No se encuentra el nombre del origen de datos y no se especificó ningún controlador predeterminado (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)

In [ ]:
df.to_sql("Licitaciones", engine)
engine.dispose()

In [16]:
driver.close()

In [ ]:
import zipfile, os

PATH = "archives/"

with zipfile.ZipFile(PATH+"Expediente.zip", mode="w") as archive:
    for exp in os.listdir(PATH):
        if ".png" in exp: 
            archive.write(PATH + exp)
            os.remove(PATH + exp)